In [ ]:
import pandas as pd
import numpy as np

# -----------------------------------------------------------
# 1. 데이터 로드
# -----------------------------------------------------------
# 파일 경로 설정 (실제 경로에 맞게 수정해주세요)
file_path_usage = "../../30만원본/카드승인정보_300k.csv"
file_path_balance = "../../30만원본/카드잔액정보_300k.csv"

# 분석에 필요한 한글 컬럼 리스트 정의
# (데이터 파일의 첫 줄 헤더가 이 이름들과 정확히 일치해야 합니다)
cols_usage = [
    '발급회원번호', '기준년월', 
    '이용개월수_전체_R3M', '이용개월수_전체_R6M'
]

cols_balance = [
    '발급회원번호', '기준년월',
    # 악성부채 관련
    '잔액_현금서비스_B0M', '잔액_현금서비스_B1M', 
    '잔액_카드론_B0M', '잔액_카드론_B1M', 
    # 연체 관련
    '연체잔액_B0M', '연체원금_B1M', '연체원금_B2M',
    # 자산 관련
    '평잔_3M', '평잔_6M'
]

# CSV 파일 로드
# usecols를 사용하여 필요한 컬럼만 로드 (메모리 절약)
try:
    df_usage = pd.read_csv(file_path_usage, usecols=lambda x: x in cols_usage)
    df_balance = pd.read_csv(file_path_balance, usecols=lambda x: x in cols_balance, encoding= 'cp949')
    print("데이터 로드 완료")
except ValueError as e:
    print("오류 발생: 컬럼명이 파일과 일치하지 않습니다. 파일의 헤더를 확인해주세요.")
    print(e)

# -----------------------------------------------------------
# 2. 데이터 병합 (Merge)
# -----------------------------------------------------------
# 발급회원번호, 기준년월 기준으로 병합
df = pd.merge(df_usage, df_balance, on=['발급회원번호', '기준년월'], how='inner')

# 결측치는 0으로 채움 (계산 오류 방지)
df.fillna(0, inplace=True)

# -----------------------------------------------------------
# 3. 핵심 지표 산출
# -----------------------------------------------------------

# [부정 1] 점수_악성부채 (Bad Debt Score)
# : (당월 - 전월) / (전월+1) -> 급증할수록 점수 상승
# 현금서비스 가중치 1.5, 카드론 가중치 1.0
df['점수_악성부채'] = (
    ((df['잔액_현금서비스_B0M'] - df['잔액_현금서비스_B1M']) / (df['잔액_현금서비스_B1M'] + 1) * 1.5) +
    ((df['잔액_카드론_B0M'] - df['잔액_카드론_B1M']) / (df['잔액_카드론_B1M'] + 1) * 1.0)
)

# [부정 2] 점수_연체강도 (Delinquency Score)
# : 최근 연체일수록 가중치 높음 (당월 3배, 전월 2배, 전전월 1배)
df['점수_연체강도'] = (
    (3.0 * df['연체잔액_B0M']) + 
    (2.0 * df['연체원금_B1M']) + 
    (1.0 * df['연체원금_B2M'])
)

# [긍정 1] 점수_활동성 (Activity Score)
# : (최근3개월*2) - 6개월치 -> 양수면 최근 활동이 더 활발함 (이탈확률 낮음)
df['점수_활동성'] = (
    ((df['이용개월수_전체_R3M'] * 2) - df['이용개월수_전체_R6M']) / (df['이용개월수_전체_R6M'] + 1) * 100
)

# [긍정 2] 점수_자산방어 (Asset Score)
# : 3개월평잔 / 6개월평잔 -> 1보다 크면 자산 유지/증가 (이탈확률 낮음)
df['점수_자산방어'] = (
    df['평잔_3M'] / (df['평잔_6M'] + 1) * 10
)

# -----------------------------------------------------------
# 4. 종합 이탈 점수 (Total Churn Score) 산출
# -----------------------------------------------------------
# 공식: (부정 점수 합) - (긍정 점수 합)
# 점수가 높을수록 이탈 위험이 높음
df['종합이탈점수'] = (
    (df['점수_악성부채'] + df['점수_연체강도']) - 
    (df['점수_활동성'] + df['점수_자산방어'])
)

# -----------------------------------------------------------
# 5. 최종 결과 정리 (df_final)
# -----------------------------------------------------------
final_cols = [
    '발급회원번호', '기준년월', 
    '점수_악성부채', '점수_연체강도',  # (+) 이탈 요인
    '점수_활동성', '점수_자산방어',    # (-) 유지 요인
    '종합이탈점수'                    # Final Target Score
]

df_final = df[final_cols].copy()

# 결과 확인
df_final.head()

# -----------------------------------------------------------
# 6. CSV 저장 (주석 처리됨)
# -----------------------------------------------------------
# df_final.to_csv("종합이탈점수_산출결과.csv", index=False, encoding='utf-8-sig')